In [4]:
catalog


'm3_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'reflex_stock_dataset': kedro_datasets.pandas.sql_dataset.SQLQueryDataset
'm3_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'reflex_stock_parquet': kedro_datasets.pandas.parquet_dataset.ParquetDataset
'parameters': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules': kedro.io.memory_dataset.MemoryDataset(data='<dict>')
'params:reflex_mapping_rules.STD': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.CAT': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.QUA': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.REC': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.RET': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.BLO': kedro.io.memory_dataset.MemoryDataset(data='<str>')
'params:reflex_mapping_rules.DEF

In [5]:
m3_stock = catalog.load('m3_stock_parquet')

[12/09/25 13:16:05] INFO     Loading data from m3_stock_parquet (ParquetDataset)...            ]8;id=723424;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=961143;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [6]:
m3_stock.where(m3_stock['SKU_M3'] == "70AAM76JNI090E").dropna()

,SKU_M3,Depot,Emplacement,Lot,Quantite,WMS
3529,70AAM76JNI090E,150,STOCK,1000002362,10.0,N/A
4578,70AAM76JNI090E,100,STOCK,1000002362,374.0,N/A
26172,70AAM76JNI090E,100,RETOUR,OI0500059560,1.0,N/A
27967,70AAM76JNI090E,100,RETOUR,OI0500059580,5.0,N/A


In [7]:
rfx_stock = catalog.load("reflex_stock_parquet")

[12/09/25 13:16:06] INFO     Loading data from reflex_stock_parquet (ParquetDataset)...        ]8;id=835746;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=899018;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [8]:
rfx_stock.where(rfx_stock['SKU'] == "70AAM76JNI090E").dropna()

,SKU,Qualite_Origine,Emplacement,Stock_en_VL,Lot_1
3451,70AAM76JNI090E,RET,NDISP,1.0,1000002053
10536,70AAM76JNI090E,STD,STOCK,384.0,1000002362


In [9]:
corr_df = catalog.load('reflex_m3_correspondence')

                    INFO     Loading data from reflex_m3_correspondence (ParquetDataset)...    ]8;id=777896;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=785429;file:///home/mdelapintiere/ds/regulstock/.venv/lib/python3.12/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [ ]:
corr_df.where(corr_df['lot'] == 'SWF23-0051').dropna()

,sku,lot,reflex_emplacement,reflex_category,m3_depot,m3_emplacement,mapping_rule,qty_reflex,qty_m3
0,BD50XAJ99ECH0002,SWF23-0051,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",9.0,0.0
1,BD50XAJ99ECH0002,SWF23-0051,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",9.0,0.0
2,BD50XAJ99ECH0002,SWF23-0051,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",9.0,0.0
3,SC70A5640575,WF24-0159B,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",22.0,0.0
4,SC70A5640575,WF24-0159B,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",22.0,0.0
...,...,...,...,...,...,...,...,...,...
58160,SC70A4438000V,WF22-0294,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",1.0,0.0
58161,SC70A4438000V,WF22-0294,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",1.0,0.0
58162,SC70A4438000V,WF22-0294,STOCK,STOCK,None,None,"STOCK -> WHLO ['100', '150', '400'] / WHSL STOCK",1.0,0.0
58163,40PAN76JRK4648,1000002577,NDISP,NDISP,100,NDISP,"NDISP -> WHLO ['100', '400'] / WHSL NDISP",1.0,1.0


In [20]:
corr_df.where(corr_df['lot'] == 'SWF23-0051').dropna()

,sku,lot,reflex_emplacement,reflex_category,m3_depot,m3_emplacement,mapping_rule,qty_reflex,qty_m3


In [21]:
m3_stock.where(m3_stock.Lot == '1000002577').dropna()

,SKU_M3,Depot,Emplacement,Lot,Quantite,WMS
609,70AAN76JRK085F,150,STOCK,1000002577,15.0,N/A
990,70AAN76JRK095E,100,STOCK,1000002577,226.0,N/A
1013,40PAN76JRK4648,100,NDISP,1000002577,1.0,N/A
1720,70AAN76JRK090D,150,STOCK,1000002577,10.0,N/A
2213,40PAN76JRK4244,100,STOCK,1000002577,319.0,N/A
2566,60PAN76JRK4244,100,STOCK,1000002577,524.0,N/A
3706,40PAN76JRK4648,150,STOCK,1000002577,10.0,N/A
4363,70AAN76JRK090F,100,STOCK,1000002577,5.0,N/A
5196,60PAN76JRK5052,150,STOCK,1000002577,11.0,N/A
5598,70AAN76JRK085D,100,STOCK,1000002577,352.0,N/A
